In [1]:
import json
import time
import requests

# google libraries
import googlemaps
import polyline

# mapping and shape utils
import folium
from folium import plugins

# data processing
import pandas as pd

In [2]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [3]:
directions = []
with open('outputs/directions_dict_5.31.2022.jsonl') as f:
    for line in f:
        d_ = json.loads(line.split('\n')[0])
        directions.append(d_)
        
routes = pd.read_csv('outputs/selected_routes.csv')
results = pd.read_csv('outputs/linear_prog_results.csv')

In [4]:
results_filtered = results[results['value']>0][['conflict','crossing','mode','value']]

In [5]:
results_filtered = pd.merge(results_filtered,routes,how='left',left_on=['conflict','crossing','mode'],
                                   right_on=['conflict','crossing','mode'])

In [6]:
dirs = set()
for d in directions:
    dirs.add(f"{d['conflict']}_{d['crossing']}")
    
dirs_res = set()
for kk, d in results_filtered.iterrows():
    dirs_res.add(f"{d['conflict']}_{d['crossing']}")    

In [7]:
directions_filtered = []

# filter out routes that were not selected
for d in directions:
    match = results[(results.conflict==d['conflict'])\
                    &(results.crossing==d['crossing'])\
                    &(results['mode']==d['mode'])]\
                    .iloc[0]['value']
    d['results'] = match
    if match > 0:
        directions_filtered.append(d)

In [8]:
print(f"Of the {len(directions)} total possible routes, only {len(directions_filtered)} were selected by the linear program")

Of the 406 total possible routes, only 52 were selected by the linear program


In [9]:
c_desc = results[results['value']>0]['value'].describe()
def bucket_count(count):
    if count <= c_desc['25%']:
        stroke = 1.5
    elif count <= c_desc['50%']:
        stroke = 3.5
    elif count <= c_desc['75%']:
        stroke = 5.5
    else:
        stroke = 7
    return stroke   

In [10]:
def get_refugees(conflict, crossing):
    match = results[(results.conflict==conflict) & (results.crossing==crossing)].iloc[0].value
    return int(match)

In [11]:
def polyline_to_geojson(d):
    polyline_ = polyline.decode(d['directions'][0]['overview_polyline']['points'])
    # reverse order to comply with geojson spec
    coords_list = [[lon, lat] for lat, lon in polyline_]
    
    feature = ({
        "type": "Feature",
        "properties": {
            "Conflict": d['conflict'],
            "Crossing": d['crossing'],
            "Crossing_country": d['crossing_country'],
            "Mode": d['mode'],
            "Refugees": get_refugees(d['conflict'], d['crossing']),
            "Duration": d['directions'][0]['legs'][0]['duration']['text']
        },
        "geometry": {
            "type": "MultiLineString",
            "coordinates": [coords_list]
        }
    })
    return feature

def style_function(feature):

    if feature['properties']['Mode'] == 'transit':
        color = '#7570b3'
    else:
        color = '#4A89F3'         
    
    stroke = bucket_count(feature['properties']['Refugees'])
    
    return {
        "weight": stroke,
        "color": color,
        "opacity": 0.7
    }

def highlight_function(feature):
    
    return {
        "weight": 12,
        "color": 'red',
        "opacity": 1
    }

In [12]:
# Create Map
map = folium.Map(location=[routes.conflict_lat.mean(),routes.conflict_lon.mean()], zoom_start=6)

In [13]:
# Plot conflict starting points
for kk, vv in results_filtered.iterrows():
    start_m = folium.Marker([vv.conflict_lat, vv.conflict_lon], popup=vv.conflict, 
                            icon=folium.Icon(icon='glyphicon glyphicon-fire', color='darkred'))
    start_m.add_to(map)
    
# Plot ending locations
for kk, vv in results_filtered.iterrows():
    icon = 'glyphicon glyphicon-road'
    color = 'orange'
    popup_text = f"<b>Crossing Name: </b>{vv.crossing}<br>"\
                 f"<b>Country: </b>{vv.crossing_country}"
    popup = folium.Popup(popup_text, max_width=300,min_width=150)
    xing = folium.Marker([vv.crossing_lat, vv.crossing_lon], popup=popup, 
                          icon=folium.Icon(icon=icon, color=color))
    xing.add_to(map)

In [14]:
fg = folium.FeatureGroup("Routes")
for d in directions:
    refugees = get_refugees(d['conflict'], d['crossing'])
    if refugees > 0:
        distance = d['directions'][0]['legs'][0]['distance']['text']
        duration = d['directions'][0]['legs'][0]['duration']['text']
        tooltip = f"Travel between <b>{d['conflict']}</b> and <b>{d['crossing']}, {d['crossing_country']}"\
                  f"</b> by {d['directions'][0]['mode']} is <b>"\
                  f"{distance}</b> and takes <b>{duration}</b>.\n"\
                  f"{refugees} total refugees are expected to take this route."
        tooltip = folium.GeoJsonTooltip(fields=['Conflict','Crossing','Mode','Refugees', 'Duration'])
        feat = polyline_to_geojson(d)
        polyline_m = folium.GeoJson(feat, tooltip=tooltip, style_function=style_function, 
                                    highlight_function=highlight_function, zoom_on_click=True)

        polyline_m.add_to(fg)

fg.add_to(map)

# Add custom basemaps
basemaps['Google Satellite Hybrid'].add_to(map)
basemaps['Google Maps'].add_to(map)

# Add a layer control panel to the map.
map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(map)

In [15]:
display(map)

In [16]:
# save map
map.save('mapping/results_map.html')